In [22]:
import sys
import os
from pathlib import Path
from openfermion.hamiltonians import MolecularData
from openfermion.transforms import get_fermion_operator, get_interaction_operator
from openfermionpsi4 import run_psi4

add functions to automate the generation of hamiltonians with varying
number of active spaces, choice of mapping, choice of basis representation.

Getting this to run is a real pain in the neck...
1. Do a fresh conda install (openfermionpsi4 is specifically developed for anaconda apparently)
2. Install openfermion: conda install openfermion
3. Install psi4 and pip: (from the openfermionpsi4 github)
4. Install openfermionpsi4: (also from the openfermionpsi4 github

I found that even when I used a virtual environment, calling run_psi4() would still reach
out to the installed anaconda version resulting in seg faults that crashed the program.

https://github.com/quantumlib/OpenFermion-Psi4

In [23]:
def generate_and_save(geometry, basis, multiplicity, description, mfilename):
    # initialize the molecule
    molecule = MolecularData(geometry,basis,multiplicity,description=description,filename=mfilename)
    molecule.save()
    
    # compute the active space integrals
    print('-computing integrals-')
    molecule = run_psi4(molecule,run_mp2=True,run_cisd=True,run_ccsd=True,run_fci=True)
    
    print(molecule.filename)
    print(molecule.two_body_integrals)
    print(molecule.canonical_orbitals)
    molecule.save()    
    print('Successful generation')

In [32]:
def load_and_transform(filename, orbitals, transform):
    # Load data
    print('--- loading molecule ---')
    molecule = MolecularData(filename=filename)
    
    print('filename: {}'.format(molecule.filename))
    #print('n_atoms: {}'.format(molecule.n_atoms))
    #print('n_electrons: {}'.format(molecule.n_electrons))
    #print('n_orbitals: {}'.format(molecule.n_orbitals))
    #print('Canonical Orbitals: {}'.format(molecule.canonical_orbitals))
    #print('n_qubits: {}'.format(molecule.n_qubits))
    
    # get the Hamiltonian for a specific choice of active space
    # set the Hamiltonian parameters
    occupied_orbitals, active_orbitals = orbitals
    
    molecular_hamiltonian = molecule.get_molecular_hamiltonian(
                 occupied_indices=range(occupied_orbitals),
                 active_indices=range(active_orbitals))
    
    # map the operator to fermions and then qubits
    fermion_hamiltonian = get_fermion_operator(molecular_hamiltonian)
    
    # get interaction operator
    interaction_hamiltonian = get_interaction_operator(fermion_hamiltonian)
    
    if transform is 'JW':
        qubit_h = jordan_wigner(fermion_hamiltonian)
        qubit_h.compress()
    elif transform is 'BK':
        qubit_h = bravyi_kitaev(fermion_hamiltonian)
        qubit_h.compress()
    elif transform is 'BKSF':
        qubit_h = bravyi_kitaev_fast(interaction_hamiltonian)
        qubit_h.compress()
    elif transform is 'BKT':
        qubit_h = bravyi_kitaev_tree(fermion_hamiltonian)
        qubit_h.compress()
    elif transform is 'PC':
        qubit_h = binary_code_transform(fermion_hamiltonian, parity_code(2*active_orbitals))
        qubit_h.compress()
    else:
        print('ERROR: Unrecognized qubit transformation: {}'.format(transform))
        sys.exit(2)
        
    return qubit_h

In [33]:
def write_to_file(filename, name, Ne, hamiltonian, description):
    # write the resulting qubit H to file
    print('\n\n~~ writing Qubit Hamiltonian to file~~\n')
    print('filename: {}'.format(filename))
    with open(filename, 'w') as H_file:
        H_file.write('{} {}\n'.format(name, Ne))
        hstring = '{}'.format(hamiltonian)
        terms = hstring.split('\n')
        for t in terms:
            t2 = t.split('[')
            if len(t2) is 2:
                coef = t2[0]
                paul = t2[1].split(']')[0]
                # Check for identity operator
                if paul is '':
                    paul = 'I0'
                    
                # Write coefficients and operators to file
                H_file.write('{0:17s} {1}\n'.format(coef,paul))
                
            else:
                print('ERROR: Something went wrong parsing string')
    print('Successful write\n')

In [34]:
from openfermion.transforms import (jordan_wigner, bravyi_kitaev, bravyi_kitaev_fast, bravyi_kitaev_tree,
                                    binary_code_transform, reverse_jordan_wigner)
from openfermion.transforms._binary_codes import parity_code

In [35]:
# Set molecule parameters for H2
name = 'H2'
basis = '6-31g'
multiplicity = 1
mult = 'singlet'
num_electrons = 2
bond_length = 0.7
description = str(bond_length)
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]

# set Active Space parameters
occupied_num = 4
active_num = 4

molecule_file = 'molecule_data/{}_{}_{}_{}'.format(name,basis,mult,bond_length)

print('--- Generate Molecule: {}_{}_{:.2f} ---'.format(name,basis,bond_length))

generate_and_save(geometry, basis, multiplicity, description, molecule_file)
                
# Load the molecule and perform qubit transformation
orbitals = (occupied_num, active_num)

for transform in ['JW','BK','BKSF','BKT','PC']:
    qubit_h = load_and_transform(molecule_file, orbitals, transform)

    # Write the qubit hamiltonian to file
    folder = 'hamiltonians/'
    if not os.path.isdir(folder):
        os.makedirs(folder)
    fn = '{}_{}_{}_{}_AS{}.txt'.format(name,basis,transform,
                                       description,active_num)
    write_to_file(folder+fn,name,num_electrons,qubit_h,description)

--- Generate Molecule: H2_6-31g_0.70 ---
-computing integrals-
molecule_data/H2_6-31g_singlet_0.7
[[[[ 6.65568116e-01 -4.09897420e-16  1.72481041e-01 -9.43463597e-17]
   [-3.50482870e-16  7.56290268e-02 -2.19696837e-16  7.83652828e-02]
   [ 1.72481041e-01 -3.78998610e-16  1.11349784e-01 -5.27324025e-17]
   [-2.13324180e-16  7.83652828e-02 -2.00335938e-16  1.41915189e-01]]

  [[-3.13789258e-16  7.56290268e-02 -1.15704121e-16  7.83652828e-02]
   [ 4.32259725e-01 -3.12376482e-16  4.70974913e-02  3.97206681e-16]
   [-3.05145881e-16 -2.26170781e-02  1.03988768e-16  1.72287632e-02]
   [ 1.43379612e-01  9.42023404e-16  7.12140507e-02 -7.86004548e-16]]

  [[ 1.72481041e-01 -9.58625462e-17  1.11349784e-01 -1.32887885e-16]
   [-2.02327927e-16 -2.26170781e-02  3.48416786e-16  1.72287632e-02]
   [ 5.36196364e-01 -8.36217084e-17  1.19500314e-01  8.63088187e-17]
   [-2.70499158e-16  7.73486003e-02 -3.82725571e-16  1.21478253e-01]]

  [[-2.53452603e-16  7.83652828e-02 -2.99693717e-16  1.41915189e-01]

In [39]:
# Set molecule parameters for H2O
name = 'H2O'
basis = '6-31g'
multiplicity = 1
mult = 'singlet'
num_electrons = 8
description = '104'
geometry = [('O', (0.,0.,0.)), ('H', (0.757,0.586,0.)), ('H', (-0.757,0.586,0.))]

# set Active Space parameters
occupied_num = 6
active_num = 6

molecule_file = 'molecule_data/{}_{}_{}_{}'.format(name,basis,mult,description)

print('--- Generate Molecule: {}_{}_{} ---'.format(name,basis,description))

generate_and_save(geometry, basis, multiplicity, description, molecule_file)
                
# Load the molecule and perform qubit transformation
orbitals = (occupied_num, active_num)

for transform in ['JW','BK','BKSF','BKT','PC']:
    qubit_h = load_and_transform(molecule_file, orbitals, transform)

    # Write the qubit hamiltonian to file
    folder = 'hamiltonians/'
    if not os.path.isdir(folder):
        os.makedirs(folder)
    fn = '{}_{}_{}_{}_AS{}.txt'.format(name,basis,transform,
                                       description,active_num)
    write_to_file(folder+fn,name,num_electrons,qubit_h,description)

--- Generate Molecule: H2O_6-31g_104 ---
-computing integrals-
molecule_data/H2O_6-31g_singlet_104
[[[[ 4.73965674e+00  4.27906754e-01 -2.40990004e-15 ...  1.82181397e-01
     9.92243176e-15  3.55267643e-01]
   [ 4.27906754e-01  6.32193143e-02 -2.53008401e-16 ...  2.21140377e-02
     1.60621528e-15  5.30721295e-02]
   [-2.39375882e-15 -2.51612529e-16  1.94351471e-02 ... -1.08324941e-15
     3.18946665e-02 -4.49993133e-16]
   ...
   [ 1.82181397e-01  2.21140377e-02 -1.08464924e-15 ...  3.42368605e-02
    -1.00795479e-15  1.64809426e-02]
   [ 1.00184162e-14  1.62660853e-15  3.18946665e-02 ... -9.96419566e-16
     5.26300575e-02  9.35078169e-16]
   [ 3.55267643e-01  5.30721295e-02 -4.53702459e-16 ...  1.64809426e-02
     9.44598418e-16  4.50453429e-02]]

  [[ 4.27906754e-01  6.32193143e-02 -2.53232162e-16 ...  2.21140377e-02
     1.61616377e-15  5.30721295e-02]
   [ 1.04240564e+00  1.31773467e-02 -3.26429381e-16 ...  1.60683959e-02
     6.36297478e-17  9.56662317e-03]
   [-1.00472110e-15 



~~ writing Qubit Hamiltonian to file~~

filename: hamiltonians/H2O_6-31g_JW_104_AS6_8e.txt
Successful write

--- loading molecule ---
filename: molecule_data/H2O_6-31g_singlet_104


~~ writing Qubit Hamiltonian to file~~

filename: hamiltonians/H2O_6-31g_BK_104_AS6_8e.txt
Successful write

--- loading molecule ---
filename: molecule_data/H2O_6-31g_singlet_104


~~ writing Qubit Hamiltonian to file~~

filename: hamiltonians/H2O_6-31g_BKSF_104_AS6_8e.txt
Successful write

--- loading molecule ---
filename: molecule_data/H2O_6-31g_singlet_104


~~ writing Qubit Hamiltonian to file~~

filename: hamiltonians/H2O_6-31g_BKT_104_AS6_8e.txt
Successful write

--- loading molecule ---
filename: molecule_data/H2O_6-31g_singlet_104


~~ writing Qubit Hamiltonian to file~~

filename: hamiltonians/H2O_6-31g_PC_104_AS6_8e.txt
Successful write

